In [1]:
from embedbase.settings import get_settings_from_file
settings = get_settings_from_file()

import openai

openai.api_type = settings.api_type
openai.api_key = settings.azure_api_key
openai.api_base = settings.azure_api_base
openai.api_version = settings.azure_api_version
deployment_id = settings.azure_deployment_id
openai.Deployment.list()

<OpenAIObject list at 0x7f3b8004a3e0> JSON: {
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "ada002-1",
      "status": "succeeded",
      "created_at": 1694591112,
      "updated_at": 1694591112,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt-gpt35-1",
      "status": "succeeded",
      "created_at": 1694591175,
      "updated_at": 1694591175,
      "object": "deployment"
    }
  ],
  "object": "list"
}

In [2]:
embeddings = openai.Embedding.create(deployment_id=deployment_id,
                                     input="Hello World")
len(embeddings['data'][0]['embedding'])

1536

In [3]:
from embedbase.embedding.openai import OpenAI
openai = OpenAI(settings=settings)
print(openai.deployment_id)
len((await openai.embed("Hello World"))[0])

ada002-1


1536

In [4]:
from embedbase.database.qdrant_db import Qdrant
qdb = Qdrant(url=settings.qdrant_url, api_key=settings.qdrant_api_key)
await qdb.get_datasets()

Qdrant collections: set()


[]

In [5]:
from embedbase import get_app

# here we use openai to create embeddings and qdrant to store the data
app = get_app(settings).use_embedder(openai).use_db(qdb).run()

2023-10-05 17:41:01,494 - embedbase - INFO - Enabling Embedder <embedbase.embedding.openai.OpenAI object at 0x7f3b80037cd0>
2023-10-05 17:41:01,496 - embedbase - INFO - Enabling Database <embedbase.database.qdrant_db.Qdrant object at 0x7f3bf82d6560>



                 _           _   _         _               _          _         
        /\ \        /\_\/\_\ _    / /\            /\ \       /\ \       
       /  \ \      / / / / //\_\ / /  \          /  \ \     /  \ \____  
      / /\ \ \    /\ \/ \ \/ / // / /\ \        / /\ \ \   / /\ \_____\ 
     / / /\ \_\  /  \____\__/ // / /\ \ \      / / /\ \_\ / / /\/___  / 
    / /_/_ \/_/ / /\/________// / /\ \_\ \    / /_/_ \/_// / /   / / /  
   / /____/\   / / /\/_// / // / /\ \ \___\  / /____/\  / / /   / / /   
  / /\____\/  / / /    / / // / /  \ \ \__/ / /\____\/ / / /   / / /    
 / / /______ / / /    / / // / /____\_\ \  / / /______ \ \ \__/ / /     
/ / /_______\/_/    / / // / /__________\/ / /_______\ \ \___\/ /      
\/__________/ _      \/_/ \/_____________/\/__________/  \/_____/       
             / /\            / /\               / /\         /\ \       
            / /  \          / /  \             / /  \       /  \ \      
           / / /\ \        / / /\ \        

In [ ]:
import uvicorn
uvicorn.run(app)